## Binary classification of bank marketing data using Spiking Neural Networks with temporal dynamics

### Importing Libraries

In [1]:
import json
import time
import datetime
import warnings
from ucimlrepo import fetch_ucirepo
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import snntorch as snn
import optuna
from optuna.trial import TrialState
warnings.filterwarnings('ignore')

C:\Users\tiago\Projects\Advanced_AI_Research\spiking_nn_projects\env_one\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Preprocessing

In [2]:
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 
  
# data (as pandas dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 
  
print(X.head())
print(y.head())
print("\nColumn names:", X.columns.tolist())
print("Data types:\n", X.dtypes)

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married        NaN      no     1506     yes   no   
4   33           NaN   single        NaN      no        1      no   no   

  contact  day_of_week month  duration  campaign  pdays  previous poutcome  
0     NaN            5   may       261         1     -1         0      NaN  
1     NaN            5   may       151         1     -1         0      NaN  
2     NaN            5   may        76         1     -1         0      NaN  
3     NaN            5   may        92         1     -1         0      NaN  
4     NaN            5   may       198         1     -1         0      NaN  
    y
0  no
1  no
2  no
3  no
4  no

Column names: ['age', 'job', 'marital', 'education', 'de

#### Identify categorical and numerical columns

In [3]:
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64']).columns.tolist()

print(f"\nCategorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")


Categorical columns: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
Numerical columns: ['age', 'balance', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']


#### Creating column transformer to pre process all collumns and a standart way

In [ ]:
column_transformer = ColumnTransformer(
    [("one-hot", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols),
     ("scaler", StandardScaler(), numerical_cols)],
    remainder='passthrough'
)

### Applying transformations

In [5]:
# Transform X
X_transformed = column_transformer.fit_transform(X)

# Transform y
label_encoder = LabelEncoder()
y_transformed = label_encoder.fit_transform(y.values.ravel())

print(X_transformed)
print(y_transformed)

[[ 0.          0.          0.         ... -0.56935064 -0.41145311
  -0.25194037]
 [ 0.          0.          0.         ... -0.56935064 -0.41145311
  -0.25194037]
 [ 0.          0.          1.         ... -0.56935064 -0.41145311
  -0.25194037]
 ...
 [ 0.          0.          0.         ...  0.72181052  1.43618859
   1.05047333]
 [ 0.          1.          0.         ...  0.39902023 -0.41145311
  -0.25194037]
 [ 0.          0.          1.         ... -0.24656035  1.4761376
   4.52357654]]
[0 0 0 ... 1 0 0]


### Export preprocessing parameters to JSON for production use

In [ ]:
preprocessing_params = {
    "column_transformer_info": {
        "categorical_columns": categorical_cols,
        "numerical_columns": numerical_cols
    },
    "one_hot_encoder": {
        "categories": [cat.tolist() for cat in column_transformer.named_transformers_['one-hot'].categories_],
        "feature_names": column_transformer.named_transformers_['one-hot'].get_feature_names_out(categorical_cols).tolist(),
        "handle_unknown": "ignore"
    },
    "standard_scaler": {
        "mean": column_transformer.named_transformers_['scaler'].mean_.tolist(),
        "scale": column_transformer.named_transformers_['scaler'].scale_.tolist(),
        "var": column_transformer.named_transformers_['scaler'].var_.tolist(),
        "feature_names": numerical_cols
    },
    "label_encoder": {
        "classes": label_encoder.classes_.tolist(),
        "class_mapping": {str(cls): int(idx) for idx, cls in enumerate(label_encoder.classes_)}
    },
    "input_dimension": int(X_transformed.shape[1]),
    "preprocessing_date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

preprocessing_filename = f"preprocessing_params_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(preprocessing_filename, 'w') as f:
    json.dump(preprocessing_params, f, indent=4)

print(f"\n Preprocessing parameters saved to: {preprocessing_filename}")
print(f"  - OneHotEncoder categories: {len(preprocessing_params['one_hot_encoder']['categories'])} categorical features")
print(f"  - StandardScaler: {len(preprocessing_params['standard_scaler']['mean'])} numerical features")


 Preprocessing parameters saved to: preprocessing_params_20251015_111305.json
  - OneHotEncoder categories: 9 categorical features
  - StandardScaler: 7 numerical features


### Data Splitting

In [7]:
# First split: separate out 60% for training, 40% for temp (dev + test)
# Stratify keeps the same ratio of "yes" and "no" in training, dev, and test sets.
X_train, X_temp, y_train, y_temp = train_test_split(
    X_transformed, y_transformed, 
    test_size=0.4,  # 40% for dev + test
    random_state=42,  # for reproducibility
    stratify=y_transformed  # maintain class distribution
)

# Second split: split the 40% into 20% dev and 20% test (50/50 split of the 40%)
# Stratify keeps the same ratio of "yes" and "no" in training, dev, and test sets.
X_dev, X_test, y_dev, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,  # 50% of 40% = 20% of total
    random_state=42,
    stratify=y_temp
)

print(X_train.shape, X_dev.shape, X_test.shape)
print(y_train.shape, y_dev.shape, y_test.shape)

(27126, 51) (9042, 51) (9043, 51)
(27126,) (9042,) (9043,)


#### Convert data to PyTorch tensors

In [8]:
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_dev_tensor = torch.FloatTensor(X_dev)
y_dev_tensor = torch.LongTensor(y_dev)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

#### Create DataLoaders

In [9]:
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### Defining Spiking Neural Network

#### Fixed Hyperparameters

In [ ]:
num_inputs = X_train.shape[1]
num_outputs = len(set(y_transformed)) # Number of unique classes (2: subscribed or not subscribed to term deposit)

#### Training Function

In [ ]:
def train_epoch(model, loader, criterion, optimizer, num_steps, epoch, num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    num_batches = len(loader)
    
    print(f"\nEpoch {epoch+1}/{num_epochs} - Training:")
    print(f"  Processing {num_batches} batches...")
    
    for batch_idx, (data, targets) in enumerate(loader):
        mem1 = torch.zeros(data.size(0), num_hidden)
        spk1 = torch.zeros(data.size(0), num_hidden)
        mem2 = torch.zeros(data.size(0), num_outputs)
        
        # Reset gradients
        optimizer.zero_grad()
        
        # Forward pass through time
        spk2_rec = []  # Record output spikes
        mem2_rec = []  # Record output membrane potentials
        
        for step in range(num_steps):
            mem1, spk1, mem2, spk2 = model(data, mem1, spk1, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)
        
        # Stack recordings
        spk2_rec = torch.stack(spk2_rec)  # [num_steps, batch_size, num_outputs]
        mem2_rec = torch.stack(mem2_rec)
        
        # Loss calculation - use final membrane potential
        loss = criterion(mem2_rec[-1], targets)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy using spike counts
        spike_count = spk2_rec.sum(dim=0)  # Sum spikes over time
        _, predicted = spike_count.max(1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        total_loss += loss.item()
        
        # Print batch progress every 10 batches or on last batch
        if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == num_batches:
            current_acc = 100 * correct / total
            current_loss = total_loss / (batch_idx + 1)
            print(f"    Batch {batch_idx+1}/{num_batches} - Loss: {current_loss:.4f}, Acc: {current_acc:.2f}%")
    
    avg_loss = total_loss / len(loader)
    avg_acc = 100 * correct / total
    print(f"  Epoch {epoch+1} Training Complete - Avg Loss: {avg_loss:.4f}, Avg Acc: {avg_acc:.2f}%")
    
    return avg_loss, avg_acc

#### Evaluating Function

In [12]:
def evaluate(model, loader, criterion, num_steps, phase="Validation"):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    print(f"  {phase}...")
    
    with torch.no_grad():
        for data, targets in loader:
            # Initialize membrane potentials and spikes
            mem1 = torch.zeros(data.size(0), num_hidden)
            spk1 = torch.zeros(data.size(0), num_hidden)
            mem2 = torch.zeros(data.size(0), num_outputs)
            
            # Forward pass through time
            spk2_rec = []
            mem2_rec = []
            
            for step in range(num_steps):
                mem1, spk1, mem2, spk2 = model(data, mem1, spk1, mem2)
                spk2_rec.append(spk2)
                mem2_rec.append(mem2)
            
            spk2_rec = torch.stack(spk2_rec)
            mem2_rec = torch.stack(mem2_rec)
            
            # Loss
            loss = criterion(mem2_rec[-1], targets)
            
            # Accuracy
            spike_count = spk2_rec.sum(dim=0)
            _, predicted = spike_count.max(1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            total_loss += loss.item()
    
    avg_loss = total_loss / len(loader)
    avg_acc = 100 * correct / total
    print(f"  {phase} Complete - Loss: {avg_loss:.4f}, Acc: {avg_acc:.2f}%")
    
    return avg_loss, avg_acc

## Define the SNN model class with trial parameters

### AI model

In [13]:
class Net(nn.Module):
    def __init__(self, num_inputs, hidden_sizes, num_outputs, beta, threshold):
        super().__init__()
        
        # Create layers dynamically based on hidden_sizes list
        self.layers = nn.ModuleList()
        self.lif_layers = nn.ModuleList()
        
        # Input to first hidden layer
        prev_size = num_inputs
        for hidden_size in hidden_sizes:
            self.layers.append(nn.Linear(prev_size, hidden_size))
            self.lif_layers.append(snn.Leaky(beta=beta, threshold=threshold))
            prev_size = hidden_size
        
        # Last hidden to output layer
        self.fc_out = nn.Linear(prev_size, num_outputs)
        self.lif_out = snn.Leaky(beta=beta, threshold=threshold)

    def forward(self, x, mem_states, spk_states):
        """
        mem_states and spk_states should be lists with one element per layer
        """
        new_mem_states = []
        new_spk_states = []
        
        current_input = x
        
        # Process through hidden layers
        for i, (fc, lif) in enumerate(zip(self.layers, self.lif_layers)):
            cur = fc(current_input)
            spk, mem = lif(cur, mem_states[i])
            new_mem_states.append(mem)
            new_spk_states.append(spk)
            current_input = spk  # Next layer's input is current layer's spike
        
        # Output layer
        cur_out = self.fc_out(current_input)
        spk_out, mem_out = self.lif_out(cur_out, mem_states[-1])
        new_mem_states.append(mem_out)
        new_spk_states.append(spk_out)
        
        return new_mem_states, new_spk_states

### Training function for one epoch

In [14]:
def train_epoch(model, loader, criterion, optimizer, num_steps, hidden_sizes, num_outputs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data, targets in loader:
        # Initialize membrane potentials and spikes for all layers
        mem_states = [torch.zeros(data.size(0), size) for size in hidden_sizes]
        mem_states.append(torch.zeros(data.size(0), num_outputs))  # Output layer
        
        spk_states = [torch.zeros(data.size(0), size) for size in hidden_sizes]
        spk_states.append(torch.zeros(data.size(0), num_outputs))  # Output layer
        
        optimizer.zero_grad()
        
        spk_out_rec = []
        mem_out_rec = []
        
        for step in range(num_steps):
            mem_states, spk_states = model(data, mem_states, spk_states)
            spk_out_rec.append(spk_states[-1])  # Last layer spikes
            mem_out_rec.append(mem_states[-1])  # Last layer membrane
        
        spk_out_rec = torch.stack(spk_out_rec)
        mem_out_rec = torch.stack(mem_out_rec)
        
        loss = criterion(mem_out_rec[-1], targets)
        loss.backward()
        optimizer.step()
        
        spike_count = spk_out_rec.sum(dim=0)
        _, predicted = spike_count.max(1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        total_loss += loss.item()
    
    return total_loss / len(loader), 100 * correct / total

### Evaluation function

In [15]:
def evaluate(model, loader, criterion, num_steps, hidden_sizes, num_outputs):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, targets in loader:
            # Initialize states for all layers
            mem_states = [torch.zeros(data.size(0), size) for size in hidden_sizes]
            mem_states.append(torch.zeros(data.size(0), num_outputs))
            
            spk_states = [torch.zeros(data.size(0), size) for size in hidden_sizes]
            spk_states.append(torch.zeros(data.size(0), num_outputs))
            
            spk_out_rec = []
            mem_out_rec = []
            
            for step in range(num_steps):
                mem_states, spk_states = model(data, mem_states, spk_states)
                spk_out_rec.append(spk_states[-1])
                mem_out_rec.append(mem_states[-1])
            
            spk_out_rec = torch.stack(spk_out_rec)
            mem_out_rec = torch.stack(mem_out_rec)
            
            loss = criterion(mem_out_rec[-1], targets)
            
            spike_count = spk_out_rec.sum(dim=0)
            _, predicted = spike_count.max(1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            total_loss += loss.item()
    
    return total_loss / len(loader), 100 * correct / total

### Optuna objective function

In [ ]:
def objective(trial):
    # Most important hyperparameter: learning rate
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-2, log=True)
    
    # Adam momentum parameters
    beta1 = trial.suggest_float('adam_beta1', 0.85, 0.98)
    beta2 = trial.suggest_float('adam_beta2', 0.95, 0.9999)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    
    # Network architecture hyperparameters
    hidden_sizes = trial.suggest_categorical('hidden_sizes', [
        [64], [128], [256],    
        [256, 128], [256, 256], [128, 128], [128, 64], [64, 64], [256, 64], [128, 32],
        [256, 256, 128], [256, 128, 64], [256, 128, 128], [128, 128, 128], [128, 128, 64],
        [128, 64, 64], [128, 64, 32], [64, 64, 64], [256, 256, 256],
        [256, 256, 128, 64], [256, 128, 128, 64], [256, 128, 64, 64], [256, 128, 64, 32], [128, 128, 128, 64], 
        [128, 128, 64, 64], [128, 128, 64, 32], [128, 64, 64, 32], [64, 64, 64, 64], [256, 256, 256, 128], 
        [256, 256, 128, 128, 64], [256, 256, 128, 64, 64], [256, 256, 128, 64, 32], [256, 128, 128, 64, 64],
        [256, 128, 128, 64, 32], [256, 128, 64, 64, 32], [256, 128, 64, 32, 32], [128, 128, 128, 64, 64], 
        [128, 128, 64, 64, 64], [128, 128, 64, 64, 32], [128, 64, 64, 64, 32], [64, 64, 64, 64, 64],
        [256, 256, 256, 128, 64], [256, 256, 256, 256, 128],
    ])
    
    # Learning rate decay parameters
    lr_patience = trial.suggest_int('lr_patience', 2, 5)
    lr_factor = trial.suggest_float('lr_factor', 0.1, 0.5)

    # Spiking Neural Network Specific Hyperparameters
    beta = trial.suggest_float('beta', 0.3, 0.9999)
    threshold = trial.suggest_float('threshold', 0.8, 15.0)
    num_steps = trial.suggest_int('num_steps', 2, 20, step=5)

    # Create data loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)
    dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model with hidden_sizes
    model = Net(num_inputs, hidden_sizes, num_outputs, beta, threshold)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), 
        lr=learning_rate,
        betas=(beta1, beta2)
    )    

    # learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max', 
        factor=lr_factor, 
        patience=lr_patience
    )
    
    # Training loop
    num_epochs = 10 
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, num_steps, hidden_sizes, num_outputs
        )
        dev_loss, dev_acc = evaluate(
            model, dev_loader, criterion, num_steps, hidden_sizes, num_outputs
        )
        
        # Report intermediate value
        trial.report(dev_acc, epoch)
        
        # Handle pruning
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return dev_acc

### Run Optuna optimization

In [17]:
# Create study
study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3)
)

[I 2025-10-15 11:13:06,040] A new study created in memory with name: no-name-2114df11-cfe2-47e3-a3a5-168f61884d90


#### Optimize

In [18]:
start_time = time.time()
start_datetime = datetime.datetime.now()
print(f"\n Optimization started at: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

#We only do three trials to save time. However this code can be used with GPUs to increase the chances of finding the best hyperparameters
study.optimize(objective, n_trials=3, timeout=None, show_progress_bar=True)

# Record end time
end_time = time.time()
end_datetime = datetime.datetime.now()
elapsed_time = end_time - start_time

print(f"\n Optimization ended at: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f" Total time elapsed: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

print(f" Average time per trial: {elapsed_time/len(study.trials):.2f} seconds")


 Optimization started at: 2025-10-15 11:13:06


Best trial: 0. Best value: 88.3543:  33%|██████████████████████████████▋                                                             | 1/3 [03:01<06:03, 181.83s/it]

[I 2025-10-15 11:16:07,877] Trial 0 finished with value: 88.35434638354346 and parameters: {'learning_rate': 6.38362778772101e-05, 'adam_beta1': 0.8696679280772431, 'adam_beta2': 0.9980169341567128, 'batch_size': 32, 'hidden_sizes': [256], 'lr_patience': 3, 'lr_factor': 0.20267435957747423, 'beta': 0.47196584568104916, 'threshold': 1.4284481225138923, 'num_steps': 7}. Best is trial 0 with value: 88.35434638354346.


Best trial: 0. Best value: 88.3543:  67%|█████████████████████████████████████████████████████████████▎                              | 2/3 [04:33<02:08, 128.60s/it]

[I 2025-10-15 11:17:39,220] Trial 1 finished with value: 88.29904888299049 and parameters: {'learning_rate': 2.249070465739675e-05, 'adam_beta1': 0.9605343209469603, 'adam_beta2': 0.9930480949108201, 'batch_size': 256, 'hidden_sizes': [128, 64, 64, 64, 32], 'lr_patience': 3, 'lr_factor': 0.31079640637625705, 'beta': 0.7903996502338146, 'threshold': 5.54708369048015, 'num_steps': 7}. Best is trial 0 with value: 88.35434638354346.


Best trial: 0. Best value: 88.3543: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [09:18<00:00, 186.02s/it]

[I 2025-10-15 11:22:24,104] Trial 2 finished with value: 88.29904888299049 and parameters: {'learning_rate': 1.5045718200634862e-06, 'adam_beta1': 0.9759209474591165, 'adam_beta2': 0.9877004283788553, 'batch_size': 128, 'hidden_sizes': [256, 128, 64, 32, 32], 'lr_patience': 4, 'lr_factor': 0.15048177623861345, 'beta': 0.41168771817638805, 'threshold': 10.987706721801285, 'num_steps': 12}. Best is trial 0 with value: 88.35434638354346.

 Optimization ended at: 2025-10-15 11:22:24
 Total time elapsed: 558.06 seconds (9.30 minutes)
 Average time per trial: 186.02 seconds


### Display best results

In [19]:
print("\nBest trial:")
trial = study.best_trial

print(f"  Value (Dev Accuracy): {trial.value:.2f}%")
print("\nBest hyperparameters:")
for key, value in trial.params.items():
    print(f"  {key}: {value}")
    
best_params = trial.params


Best trial:
  Value (Dev Accuracy): 88.35%

Best hyperparameters:
  learning_rate: 6.38362778772101e-05
  adam_beta1: 0.8696679280772431
  adam_beta2: 0.9980169341567128
  batch_size: 32
  hidden_sizes: [256]
  lr_patience: 3
  lr_factor: 0.20267435957747423
  beta: 0.47196584568104916
  threshold: 1.4284481225138923
  num_steps: 7


## Train final model with best hyperparameters

### Create data loaders with best batch size

In [20]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)
dev_loader = DataLoader(dev_dataset, batch_size=best_params['batch_size'], shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False)

### Initialize final model and train him

In [23]:
model = Net(
    num_inputs, 
    best_params['hidden_sizes'],  # ✓ Get it from the best parameters
    num_outputs, 
    best_params['beta'], 
    best_params['threshold']
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'])

In [26]:
num_epochs = 8
best_dev_acc = 0
best_epoch = 0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, 
        best_params['num_steps'], best_params['hidden_sizes'], num_outputs  # ✓ Use 'hidden_sizes'
    )
    dev_loss, dev_acc = evaluate(
        model, dev_loader, criterion, 
        best_params['num_steps'], best_params['hidden_sizes'], num_outputs  # ✓ Use 'hidden_sizes'
    )
    
    print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
    print(f"  Dev   - Loss: {dev_loss:.4f}, Acc: {dev_acc:.2f}%")
    
    if dev_acc > best_dev_acc:
        best_dev_acc = dev_acc
        best_epoch = epoch + 1
        torch.save(model.state_dict(), 'best_snn_model_optuna.pth')
        print(f"  New best model saved!")


Epoch 1/8
  Train - Loss: 0.4866, Acc: 88.30%
  Dev   - Loss: 0.3064, Acc: 88.30%
  New best model saved!

Epoch 2/8
  Train - Loss: 0.2823, Acc: 88.30%
  Dev   - Loss: 0.2653, Acc: 88.30%

Epoch 3/8
  Train - Loss: 0.2520, Acc: 88.30%
  Dev   - Loss: 0.2464, Acc: 88.31%
  New best model saved!

Epoch 4/8
  Train - Loss: 0.2414, Acc: 88.31%
  Dev   - Loss: 0.2396, Acc: 88.33%
  New best model saved!

Epoch 5/8
  Train - Loss: 0.2341, Acc: 88.30%
  Dev   - Loss: 0.2342, Acc: 88.33%

Epoch 6/8
  Train - Loss: 0.2303, Acc: 88.31%
  Dev   - Loss: 0.2316, Acc: 88.33%

Epoch 7/8
  Train - Loss: 0.2272, Acc: 88.31%
  Dev   - Loss: 0.2303, Acc: 88.32%

Epoch 8/8
  Train - Loss: 0.2238, Acc: 88.31%
  Dev   - Loss: 0.2295, Acc: 88.32%


### Evaluate on test set

In [29]:
model.load_state_dict(torch.load('best_snn_model_optuna.pth'))
test_loss, test_acc = evaluate(
    model, test_loader, criterion, 
    best_params['num_steps'], best_params['hidden_sizes'], num_outputs  # ✓ Use 'hidden_sizes'
)

print(f"\nFinal Results:")
print(f"  Best Dev Accuracy: {best_dev_acc:.2f}% (epoch {best_epoch})")
print(f"  Test Accuracy: {test_acc:.2f}%")
print(f"  Test Loss: {test_loss:.4f}")


Final Results:
  Best Dev Accuracy: 88.33% (epoch 4)
  Test Accuracy: 88.30%
  Test Loss: 0.2383


## Save results

In [31]:
results = {
    "optimization_info": {
        "n_trials": len(study.trials),
        "best_trial_number": study.best_trial.number,
        "optimization_date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    "best_hyperparameters": best_params,
    "model_architecture": {
        "num_inputs": int(num_inputs),
        "hidden_sizes": best_params['hidden_sizes'], 
        "num_outputs": int(num_outputs),
        "total_parameters": int(sum(p.numel() for p in model.parameters()))  
    },
    "final_results": {
        "best_dev_accuracy": float(best_dev_acc),
        "best_epoch": int(best_epoch),
        "test_accuracy": float(test_acc),
        "test_loss": float(test_loss)
    },
    "data_info": {
        "train_samples": int(len(train_loader.dataset)),
        "dev_samples": int(len(dev_loader.dataset)),
        "test_samples": int(len(test_loader.dataset)),
        "dataset": "Bank Marketing (UCI ML Repository ID: 222)"
    }
}

json_filename = f"optuna_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(json_filename, 'w') as f:
    json.dump(results, f, indent=4)
    
print(f"\n Results saved to: {json_filename}")


 Results saved to: optuna_results_20251015_113406.json
